# Подключение к S3 (MinIO) из Python

Используем `boto3` для подключения к MinIO (совместим с S3).



Примеры: создание бакета, загрузка  вывод содержимого, удаление


In [1]:
!pip -q install boto3

import boto3

S3_ENDPOINT = "http://minio:9000"
S3_ACCESS_KEY = "minioadmin"
S3_SECRET_KEY = "minioadmin123"
BUCKET = "demo-bucket"
TEST_KEY = "hello.txt"

s3 = boto3.client(
    "s3",
    endpoint_url=S3_ENDPOINT,
    aws_access_key_id=S3_ACCESS_KEY,
    aws_secret_access_key=S3_SECRET_KEY,
)

# Создание бакета (идемпотентно)
try:
    s3.create_bucket(Bucket=BUCKET)
    print("Bucket created")
except s3.exceptions.BucketAlreadyOwnedByYou:
    print("Bucket already exists")
except Exception as e:
    print("Create bucket warning:", e)

# Загрузка тестового файла
content = b"hello from notebook"
s3.put_object(Bucket=BUCKET, Key=TEST_KEY, Body=content)
print(f"Uploaded {TEST_KEY}")

# Чтение обратно
resp = s3.get_object(Bucket=BUCKET, Key=TEST_KEY)
print("Downloaded:", resp["Body"].read().decode("utf-8"))

# Список объектов в бакете
print("\nObjects in bucket:")
resp = s3.list_objects_v2(Bucket=BUCKET)
if resp.get("Contents"):
    for obj in resp["Contents"]:
        print(f"- {obj['Key']} ({obj['Size']} bytes)")
else:
    print("(empty)")

# Удаление объекта
s3.delete_object(Bucket=BUCKET, Key=TEST_KEY)
print(f"Deleted {TEST_KEY}")

# Проверим, что объекта нет
resp = s3.list_objects_v2(Bucket=BUCKET)
keys = {o["Key"] for o in resp.get("Contents", [])}
print("Remaining keys:", keys or "(empty)")

# При необходимости можно удалить бакет (если пустой)
# s3.delete_bucket(Bucket=BUCKET)
# print("Bucket deleted")

# Вывод всех объектов в бакете
print("\nСписок объектов в бакете:")
try:
    response = s3.list_objects_v2(Bucket=BUCKET)
    if "Contents" in response:
        for obj in response["Contents"]:
            print(
                f"- {obj['Key']} (размер: {obj['Size']} байт, изменен: {obj['LastModified']})"
            )
    else:
        print("Бакет пуст")
except Exception as e:
    print(f"Ошибка при получении списка объектов: {e}")

# Вывод информации о бакете
print(f"\nИнформация о бакете '{BUCKET}':")
try:
    bucket_location = s3.get_bucket_location(Bucket=BUCKET)
    print(f"Регион: {bucket_location.get('LocationConstraint', 'us-east-1')}")
except Exception as e:
    print(f"Ошибка при получении информации о бакете: {e}")


Bucket created
Uploaded hello.txt
Downloaded: hello from notebook

Objects in bucket:
- hello.txt (19 bytes)
Deleted hello.txt
Remaining keys: (empty)

Список объектов в бакете:
Бакет пуст

Информация о бакете 'demo-bucket':
Регион: None
